In [3]:
from collections import defaultdict

In [30]:
def read_input(filename):
    with open(filename, 'r') as f:
        rows = f.readlines()
    n_rows = len(rows)
    n_cols = len(rows[0])
    d = {}
    for i in range(2, n_rows - 2):
        for j in range(2, n_cols - 3):
            c = rows[i][j]
            if c == '.':
                if rows[i-1][j].isalpha():
                    c = rows[i-2][j] + rows[i-1][j]
                elif rows[i+1][j].isalpha():
                    c = rows[i+1][j] + rows[i+2][j]
                elif rows[i][j-1].isalpha():
                    c = rows[i][j-2] + rows[i][j-1]
                elif rows[i][j+1].isalpha():
                    c = rows[i][j+1] + rows[i][j+2]
            elif c == ' ' or c.isalpha():
                c = '#'
            d[(i-2, j-2)] = c
    return d, n_rows - 4, n_cols - 5

def get_ports(maze):
    ports = defaultdict(list)
    for key, val in maze.items():
        if val.isupper():
            ports[val].append(key)
    return ports

def is_outer_port(n_rows, n_cols, v):
    return v[0] == 0 or v[1] == 0 or v[0] == n_rows - 1 or v[1] == n_cols - 1

def teleport(maze, ports, n_rows, n_cols, v):
    i, j, level = v[:3]
    w1, w2 = ports[maze[(i, j)]]
    w = w1 if v[:2] == w2 else w2
    d = -1 if is_outer_port(n_rows, n_cols, (i, j)) else 1
    return (w[0], w[1], level + d)

def get_adjacent(maze, ports, n_rows, n_cols, v):
    adj = []
    i, j, level = v[:3]
    if maze[(i, j)].isupper():
        if  (level > 0) or (level == 0 and not is_outer_port(n_rows, n_cols, (i, j))):
            q = teleport(maze, ports, n_rows, n_cols, v)
            adj.append(q)
    for ii, jj in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
        if ii < 0 or ii > n_rows - 1 or jj < 0 or jj > n_cols - 1:
            continue
        c = maze[(ii, jj)]
        if level == 0:
            if c in ('.', 'AA', 'ZZ') or (c.isupper() and not is_outer_port(n_rows, n_cols, (ii, jj))):
                adj.append((ii, jj, level))
        else:
            if (c == '.') or (c.isupper() and c not in ('AA', 'ZZ')):
                adj.append((ii, jj, level))
    return adj

def bfs(maze, n_rows, n_cols, ports):
    p_start = ports['AA'][0]
    p_goal = ports['ZZ'][0]
    queue = []
    explored = set()
    explored.add((p_start[0], p_start[1], 0))
    queue.append((p_start[0], p_start[1], 0, 0))
    while len(queue) > 0:
        v = queue.pop(0)
        if v[:2] == p_goal and v[2] == 0:
            return v[3]
        for w in get_adjacent(maze, ports, n_rows, n_cols, v):
            if w not in explored:
                explored.add(w)
                queue.append((w[0], w[1], w[2], v[3] + 1))
    return None

def runit(filename):
    maze, n_rows, n_cols = read_input(filename)
    ports = get_ports(maze)
    path_length = bfs(maze, n_rows, n_cols, ports)
    return path_length

In [31]:
runit('20_input.txt')

6492